#  Data Wrangling and Analyzing on "We Rate Dogs" Twitter Site
by Dilupa Chandrasekara

## Data Gathering

1. Twitter Archive File : Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)
2. The tweet image predictions - what breed of dog (or other things) is present in each tweet according to a neural network(Previously done and tested). This file (image_predictions.tsv) is hosted on Udacity's servers and needed to be downloaded programmatically using the Requests library and the following URL -https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

3. Twitter API & JSON - Each tweet's found in the We rate dogs we retweet count and favorite ("like") count at minimum. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

1. Twitter Archive File

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests 
import seaborn as sns
import time
import json
import tweepy


In [2]:
#Read CSV file 
df = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

2. Uses the Requests library to download the tweet image prediction (image_predictions.tsv)

In [5]:
import requests
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
data = requests.get(url)
with open (url.split('/')[-1], mode='wb') as file:
    file.write(data.content)

In [27]:
image_data = pd.read_csv('image-predictions.tsv', sep='\t')
image_data.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Uses the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [7]:
## !pip install  --upgrade tweepy==4.6

In [24]:
import tweepy

bearer_token='AAAAAAAAAAAAAAAAAAAAAMPpYwEAAAAAHYmhw4BflEGucKBU4ZBlQFjMVn0%3DMMIPovZjtqjof7qGLBpNZAOus8JxB6wky7w4Hba08CMIOQw2Nu'
# create instance
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# fields to return in query
tweet_fields=["id","in_reply_to_user_id","favourite_count", "retweet_count", "retweet_status","lang",
    "public_metrics","source","text"]
# query

tweet_id_list = df['tweet_id']

tweet_field = ["public_metrics"]

public_metrics_list = []

for tweet_id in tweet_id_list:
    temp = client.get_tweet(id=tweet_id, tweet_fields =tweet_field)
    tweet_data = temp.data
    for key in tweet_field:
        if tweet_data and tweet_data.get(key):
            public_metrics = tweet_data[key]
            public_metrics_list.append ([tweet_id, public_metrics])
        else:
            print(f'Empty tweet: {tweet_id}, tweet info: {tweet_data}')
        
    df_public_metrics = pd.DataFrame(public_metrics_list, columns = ['tweet_id', 'public_metrics'])


Empty tweet: 888202515573088257, tweet info: None
Empty tweet: 873697596434513921, tweet info: None
Empty tweet: 872668790621863937, tweet info: None
Empty tweet: 872261713294495745, tweet info: None
Empty tweet: 869988702071779329, tweet info: None
Empty tweet: 866816280283807744, tweet info: None
Empty tweet: 861769973181624320, tweet info: None
Empty tweet: 856602993587888130, tweet info: None
Empty tweet: 856330835276025856, tweet info: None
Empty tweet: 851953902622658560, tweet info: None
Empty tweet: 851861385021730816, tweet info: None
Empty tweet: 845459076796616705, tweet info: None
Empty tweet: 844704788403113984, tweet info: None


Rate limit exceeded. Sleeping for 770 seconds.


KeyboardInterrupt: 

In [25]:
##temp = []
##import json 

##with open('tweet_json.txt', encoding='utf-8') as js_file:
  ##  for i in js_file:
    ##     data = json.loads(i)
     ##    tweet_id = data['id']
      ##   tweet = data['full_text']
     ##    url1 = tweet[tweet.find('https'):]
       ##  favorite_count = data['favorite_count']
       ##  retweet_count = data['retweet_count']  
        ## retweet_status = data['retweet_status'] = data.get('retweet_status', 'Original tweet')
        ## if retweet_status == 'Original tweet':
       ##      url = url1
        ## else:
    ##        retweeted_status = 'A retweet'
          ##   url = 'A retweet'
        
    
       ##  temp.append({'tweet_id': tweet_id,'favorite_count': favorite_count,'retweet_status': retweet_status,
                     'url': url,'retweet_count': retweet_count})

# create a new DataFrame 
 ##df_new = pd.DataFrame(temp, columns = ['tweet_id', 'favorite_count','retweet_count', 'retweet_status', 'url'])
                                          
                                                           
 ##df_new.head()

 ##df_new.to_csv('tweet_json.txt', index=False)

In [29]:
dfj = pd.read_csv('tweet_json.txt')

In [31]:
dfj.head(10)

,tweet_id,favorite_count,retweet_count,retweet_status,url


In [33]:
df_public_metrics.head(10)

,tweet_id,public_metrics
0,892420643555336193,"{'retweet_count': 7087, 'reply_count': 144, 'l..."
1,892177421306343426,"{'retweet_count': 5344, 'reply_count': 172, 'l..."
2,891815181378084864,"{'retweet_count': 3517, 'reply_count': 110, 'l..."
3,891689557279858688,"{'retweet_count': 7288, 'reply_count': 142, 'l..."
4,891327558926688256,"{'retweet_count': 7826, 'reply_count': 193, 'l..."
5,891087950875897856,"{'retweet_count': 2634, 'reply_count': 58, 'li..."
6,890971913173991426,"{'retweet_count': 1685, 'reply_count': 51, 'li..."
7,890729181411237888,"{'retweet_count': 15933, 'reply_count': 160, '..."
8,890609185150312448,"{'retweet_count': 3659, 'reply_count': 94, 'li..."
9,890240255349198849,"{'retweet_count': 6155, 'reply_count': 145, 'l..."


In [34]:
df_public_metrics.loc[:,"public_metrics"]

0      {'retweet_count': 7087, 'reply_count': 144, 'l...
1      {'retweet_count': 5344, 'reply_count': 172, 'l...
2      {'retweet_count': 3517, 'reply_count': 110, 'l...
3      {'retweet_count': 7288, 'reply_count': 142, 'l...
4      {'retweet_count': 7826, 'reply_count': 193, 'l...
5      {'retweet_count': 2634, 'reply_count': 58, 'li...
6      {'retweet_count': 1685, 'reply_count': 51, 'li...
7      {'retweet_count': 15933, 'reply_count': 160, '...
8      {'retweet_count': 3659, 'reply_count': 94, 'li...
9      {'retweet_count': 6155, 'reply_count': 145, 'l...
10     {'retweet_count': 6211, 'reply_count': 154, 'l...
11     {'retweet_count': 4212, 'reply_count': 91, 'li...
12     {'retweet_count': 8437, 'reply_count': 175, 'l...
13     {'retweet_count': 3754, 'reply_count': 93, 'li...
14     {'retweet_count': 1897, 'reply_count': 60, 'li...
15     {'retweet_count': 4484, 'reply_count': 111, 'l...
16     {'retweet_count': 3800, 'reply_count': 81, 'li...
17     {'retweet_count': 3583, 

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [ ]:
df.head(10)

In [ ]:
image_data.head(10)

In [ ]:
dfj.head(10)

In [ ]:
df.info()

In [ ]:
image_data.info()

In [ ]:
dfj.info

In [ ]:
df.isnull().sum()

In [ ]:
dfj.isnull().sum()

In [ ]:
image_data.isnull().sum()

In [ ]:
sum(df.duplicated())




In [ ]:
sum(dfj.duplicated())

In [ ]:
sum(image_data.duplicated())

In [ ]:
image_data.describe()

In [ ]:
df.describe()

In [ ]:
df_new.describe()

In [ ]:
df.nunique()

In [ ]:
dfj.nunique()

In [ ]:
image_data.nunique()

In [ ]:
np.sort(df['rating_numerator'].unique()) 

In [ ]:
np.sort(df['rating_denominator'].unique())

In [ ]:
df['rating_denominator'].value_counts()

In [ ]:
df['rating_numerator'].value_counts()

### Quality issues
1.   Change tweet_id from an integer to a string .

2. Timestamp is not of datetime format 
3. Delete Retweets 

4. Cleaning the name column. ( Replacing invalid names like 'a','an' , 'not', 'quite', etc.. with None) 

5.  Wrong Data Type in Source Column- Source Column should be in Category DataType

6.  Wrong Data Type in img_num - It should be String
7. Remove columns no longer needed.

8. Checking for Duplicates and Deleting them

### Tidiness issues
1. Merge the dataframe twitter_archive, dataframe image_predictions, and tweet_json dataframes .
2. doggo, floofer, pupper, puppo these 4 variables shoule be combined into one  variable Dog Type.
3. Remove columns with too many missing values.
* retweeted_status_user_id
* retweeted_status_id
* retweeted_status_timestamp



## Create a copy of the all dataset for further  cleaning

In [ ]:
twitter = df.copy()
image = image_data.copy()
json = dfj.copy()

# Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

## Tidiness Problem 1

For rectifying the tidiness issue we need to merge the df twitter archive, df image_predictions , df tweet_json using python CONCAT command



In [ ]:
df2 = pd.concat([twitter, image, json], join='outer', axis=1)

In [ ]:
df2.head()


In [ ]:
df2.columns

In [ ]:
df2.shape

### Tidiness Problem 2

####  doggo, floofer, pupper, puppo these 4 variables shoule be combined into one variable Dog Type.

We can use the .EXTRACT() method for that. But because of having multiple dog stages in a row, we need to consider that an write the code.

#### Code

In [ ]:
df2.groupby(["doggo", "floofer", "pupper", "puppo"]).size().reset_index().rename(columns={0: "count"})

In [ ]:
df2.doggo.replace('None', '', inplace=True) 
df2.doggo.replace(np.NaN, '', inplace=True)
df2.floofer.replace('None', '', inplace=True) 
df2.floofer.replace(np.NaN, '', inplace=True)
df2.pupper.replace('None', '', inplace=True) 
df2.pupper.replace(np.NaN, '', inplace=True)
df2.puppo.replace('None', '', inplace=True) 
df2.puppo.replace(np.NaN, '', inplace=True)

In [ ]:
df2['dogtype'] = df2.doggo + df2.floofer + df2.pupper + df2.puppo
df2.loc[df2.dogtype == 'doggopupper', 'dog_type'] = 'doggo, pupper'
df2.loc[df2.dogtype == 'doggopuppo', 'dog_type'] = 'doggo, puppo'
df2.loc[df2.dogtype == 'doggofloofer', 'dog_type'] = 'doggo, floofer'

In [ ]:
df2.dogtype.value_counts()

In [ ]:
df2.dogtype.unique()

In [ ]:
for idx, row in df2.iterrows():
    if ('doggoNoneNoneNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'doggo'
    elif ('NoneNoneNonepuppo' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'puppo'
    elif ('NoneNonepupperNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'pupper'
    elif ('NoneflooferNoneNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = 'floofer'
    elif ('NoneNoneNoneNone' in row['dogtype']):
        df2.loc[idx, 'dog_type'] = None
    else:
        df2.loc[idx, 'dog_type'] = 'multiple'
df2.head()  

In [ ]:
df2= df2.drop(['dogtype'] , axis =1)

#### Test

We can now look at column that we created.

In [ ]:
df2.dog_type.value_counts()

Now we can use the .DROP() method to remove doggo,floofer etc... those unwanted names

In [ ]:
df2.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

Now we can see for sure

In [ ]:
df2.head()

### Tidiness Problem 3

#### Define - Remove columns with too many missing values

#### Code - We can use the .drop() method in here as well

In [ ]:
df2.drop(['retweeted_status_user_id','retweeted_status_id' 
           'retweeted_status_timestamp'], axis=1, inplace=True)
df2.head()

# Addressing the Quality Problem

## Quality Issue 1 - Change tweet_id from an integer to a string .

In [ ]:
df2['tweet_id'] = df2['tweet_id'].astype(str)

Test

In [ ]:
type(df2.iloc[0,0])

## Quality Issue 2 - Timestamp is not of datetime format

Creating the Timestamp in "Year-Month-Date" Form

In [ ]:
 #Change the 'timestamp' column to a datetime object
df2['timestamp'] = pd.to_datetime(df2['timestamp'], format = "%Y-%m-%d ")

In [ ]:
df2.head()

## Quality Issue 3- Delete Retweets

In [ ]:
df2 = df2.drop(['retweeted_status_id'] , axis=1)

Test

We can just check the column names.

In [ ]:
df2.columns

## Quality Issue 4 -  Cleaning the name column. ( Replacing invalid names like  'a','an' , 'not', 'quite', etc.. with None)


In [ ]:
df2['name'] = df2.name.replace(['None', 'a', 'an', 'very','the', 'not', 'quite', 'actually'], 'None' , inplace = True)

Test

In [ ]:
df2[df2['name']=='a', 'an', 'very', 'the', 'not','quite','actually']

## Quality Issue 5 - Wrong Data Type in Source Column- Source Column should be in Category DataType

In [ ]:
df2['source'] = df2['source'].astype('category')

Test - We can check the results using type method

In [ ]:
df2.source.dtype


## Quality Issue 6- Wrong Data Type in img_num - It should be String

In [ ]:
df2['img_num'] = df2['img_num'].astype(str)

Test

In [ ]:
type(df2.iloc[0,11])

## Quality Issue 7 -  Remove columns no longer needed

In [ ]:
df2.drop(['in_reply_to_status_id', 
                  'in_reply_to_user_id'], axis = 1, inplace=True)

Test

In [ ]:
df2.info()


## Quality Issue 8 - Checking for Duplicates and Deleting them

In [ ]:
df2.drop_duplicates(inplace = True)

Test

In [ ]:
df2[df2.duplicated()].sum()

### Storing our cleaned data into CSV File

In [ ]:
df2.to_csv('twitter_archive_master.csv', encoding='utf-8')

 ## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

Import the cleaned Dataset 

In [ ]:
 df1 = pd.read_csv('twitter_archive_master.csv')

In [ ]:
df1.head()

In [ ]:
df1.shape

## Insight 1
### Plot between Count of Retweet and Count of Favourites :

In [ ]:
plt.figure(figsize = [10,5])
bins = np.arange(df1['favorite_count'].min() , df1['favorite_count'].max() + 500, 500)
df2.favorite_count.plot(kind='hist',  bins=bins )
bins = np.arange(df1['retweet_count'].min() , df1['retweet_count'].max() + 500, 500)
df1.retweet_count.plot(kind='hist', bins=bins )

plt.xlabel('count')
plt.legend()
plt.xlim(200,30000)
plt.ylim(0,300)
plt.title( 'favorite/retweet');

## Insight 2

### Finding out the most Popular Dog Breed

In [ ]:
df2.['dog_type'].value_counts()


In [ ]:
dog_breed = df1.groupby('dog_breed').filter(lambda x: len(x) >= 25)

dog_breed['dog_breed'].value_counts().plot(kind = 'barh')
plt.title('Most Rated Dog Breed')
plt.xlabel('Count')
plt.ylabel('Breed of dog');

## Insight 3

### Plot of the Image Numbers

In [ ]:
g = df1['img_num'].value_counts()
g.plot(kind='bar')
plt.xlabel('Image No. (1,2,3,4)')
plt.ylabel('Count')
plt.title('Count of image no.');

## Insight 4 and Visualization

Proof that highest ratings do not recieve the most retweets

In [ ]:
df1.plot(x='retweet_count', y='rating', kind='scatter')
plt.xlabel('Retweet Counts')
plt.ylabel('Ratings')
plt.title('Retweet Counts by Ratings Scatter Plot')

fig = plt.gcf()
fig.savefig('output3.png',bbox_inches='tight');
